In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

The dataset comes from *Open Food Facts* and was downloaded from: https://www.kaggle.com/openfoodfacts/world-food-facts/home.
'Open Food Facts is a free, open, collbarative database of food products from around the world, with ingredients, allergens, nutrition facts and all the tidbits of information we can find on product labels. Open Food Facts is a non-profit association of volunteers.' Over 5,000 contributors 'have added 100 000+ products from 150 countries using our Android, iPhone or Windows Phone app or their camera to scan barcodes and upload pictures of products and their labels.'

# DATA UPLOAD

In [2]:
food = pd.read_csv(r'C:\Users\owner\food.tsv', delimiter = '\t', encoding='ISO-8859-1')

In [3]:
food.shape

(356027, 163)

In [4]:
food.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object', length=163)

In [5]:
pd.set_option('display.max_columns', 500)
food.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blÃ© noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,Na

# Selecting Retail Countries: US and France

For our analysis, we will only focus on food that is on the market in the US and France since these countries have the highest number of datapoints in the data set.

In [6]:
food['countries'].value_counts().head(20)

US                169789
France            106296
en:FR              16056
Suisse             12003
Deutschland         6900
EspaÃ±a             3896
United Kingdom      3363
United States       2358
en:CH               2207
en:BE               1759
en:GB               1647
Australia           1483
en:FR,France        1483
Ð Ð¾ÑÑÐ¸Ñ        1377
en:ES               1104
en:DE               1093
en:US                909
Belgique             830
en:AU                714
en:IT                660
Name: countries, dtype: int64

In [7]:
food_abbr = food[food['countries'].isin(['US', 'France', 'en:FR', 'United States', 'en:US'])]

In [8]:
food_abbr.shape

(295408, 163)

## Selecting Ingredients and The Target - Energy Value

In [9]:
food_abbr.shape

(295408, 163)

In [10]:
features_energy = food_abbr.loc[:, ['ingredients_text', 'energy_100g']]

In [11]:
features_energy = features_energy.dropna(how='any')

In [12]:
#verify that all the nan values were excluded
features_energy['ingredients_text'].isnull().sum()

0

In [13]:
features_energy['energy_100g'].isnull().sum()

0

In [14]:
features_energy.head()

,ingredients_text,energy_100g
1,"Bananas, vegetable oil (coconut oil, corn oil ...",2243.0
2,"Peanuts, wheat flour, sugar, rice flour, tapio...",1941.0
3,"Organic hazelnuts, organic cashews, organic wa...",2540.0
4,Organic polenta,1552.0
5,"Rolled oats, grape concentrate, expeller press...",1933.0


# Parsing The Ingredients Text

In [15]:
#making all word lower case
lower = []
for entry in features_energy['ingredients_text']:
    lower.append(entry.lower())

In [16]:
#removing the fullstops from the strings

no_fullstops = []
for entry in lower:
    entry = entry.replace('.', '')
    no_fullstops.append(entry)

In [17]:
#splitting strings at commas
split = []
for entry in no_fullstops:
    entry = entry.split(',')
    split.append(entry)

In [18]:
cleaned = []
for entry in split:
    for item in entry:
        item = item.split("(")
        cleaned.append(item)

In [19]:
cleaned_new = []
for entry in cleaned:
    for item in entry:
        item= item.strip(')')
        cleaned_new.append(item)

In [20]:
cleaned_newer = []
for entry in cleaned_new:
        if entry.count('[') >0:
            entry = entry.split('[') 
            cleaned_newer.append(entry)
        else:
            cleaned_newer.append(entry)

In [21]:
cleaned_newer_one = []
for entry in cleaned_newer:
    if entry.count('and/or') >0:
        entry = entry.split('and/or')
        cleaned_newer_one.append(entry)
    else:
        cleaned_newer_one.append(entry)

In [59]:
#handling the sublists and counting only unique words in single strings
sublists=[]
only_strings = []
for entry in cleaned_newer_one:
    if type(entry) ==list:
        sublists.append(entry[0:1])
        sublists.append(entry[1:])  
    else:
        only_strings.append(entry)

In [60]:
len(sublists)

96272

In [64]:
counts = []
items = []
for count, item in enumerate(only_strings):
    counts.append(count)
    items.append(item)